# Задание №1

Есть Pandas DataFrame со столбцами `[“customer_id”, “product_id”, “timestamp”]`, который содержит данные по просмотрам товаров на сайте. Есть проблема – просмотры одного `customer_id` не разбиты на сессии (появления на сайте). Мы хотим разместить сессии так, чтобы сессией считались все смежные просмотры, между которыми не более 3 минут.

Написать методом который создаст в `Pandas DataFrame` столбец `session_id` и проставит в нем уникальный `int id` для каждой сессии.

У каждого пользователя может быть по несколько сессий. Исходный `DataFrame` может быть большим – до 100 млн строк.

# Решение

основные импорты

In [5]:
import pandas as pd
import numpy as np
import datetime
import random
from tqdm import tqdm
import threading
import multiprocessing

генерируем массивы уникальных uuid для покупателей и товаров

In [6]:
import uuid

customers = []
max_unique_customers = 100
for _ in range(max_unique_customers):
    customers.append(str(uuid.uuid4()))
    
products = []
max_unique_products = 1000
for _ in range(max_unique_products):
    products.append(uuid.uuid4().hex)

создаем Pandas DataFrame со столбцами [“customer_id”, “product_id”, “timestamp”], который содержит данные по просмотрам товаров на сайте

In [ ]:
# matrix = []
# max_size = 100000000

# dt = datetime.datetime(2022, 9, 1)
# step = datetime.timedelta(seconds=5)

# for _ in tqdm(range(max_size)):
#     matrix.append([random.choice(customers), random.choice(products), dt])
#     dt += step
    
def list_append(id, count, dt, matrix):
    """
    Creates an empty list and then appends a 
    random number to the list 'count' number
    of times. A CPU-heavy operation!
    """
    for i in range(count):
        matrix.append([random.choice(customers), random.choice(products), dt])
        dt += step
    print("Ready job id", id)

size = 1000000   # Number of random numbers to add
procs = 100   # Number of processes to create
dt = datetime.datetime(2022, 9, 1)
step = datetime.timedelta(seconds=5)
matrix = []

# Create a list of jobs and then iterate through
# the number of threads appending each thread to
# the job list 
jobs = []

for i in range(0, procs):
    matrix_temp = list()
    thread = multiprocessing.Process(target=list_append, args=(i, size, dt, matrix,))
    jobs.append(thread)
    matrix += matrix_temp
    dt = datetime.datetime(2022, 9, 1) + datetime.timedelta(seconds=5*(size+1))

# Start the threads (i.e. calculate the random number lists)
for job in jobs:
    job.start()

# Ensure all of the threads have finished
for job in jobs:
    job.join()



In [6]:
columns = ["customer_id", "product_id", "timestamp"]

df = pd.DataFrame(data=matrix, columns=columns)
df

,customer_id,product_id,timestamp
0,6bd89cb1-c4e4-4bf5-a5b0-b7f3e3f55970,31ffec0e93b14293a2141832411d9990,2022-09-01 00:00:00
1,4d2b9cee-4515-49cd-b28a-eef33be92ff7,33c285bf77f641aa990a7c788fdc18c0,2022-09-01 00:00:05
2,cb50fc8e-a590-4448-968e-72d3e51300c3,caf7eb47c39f408a8e2fccbd9c007b9b,2022-09-01 00:00:10
3,4d2b9cee-4515-49cd-b28a-eef33be92ff7,4fcc26f231694568ad87ff8940070c94,2022-09-01 00:00:15
4,f37fe503-429d-41ee-ac4f-779f0348bf62,a62d22c6a1264409a5f8c1a457adbe52,2022-09-01 00:00:20
...,...,...,...
9999995,f2465618-3a39-4c0a-b5f9-b85ef097c9f6,1e19d0f4169245a49f3257aa92414175,2022-09-12 13:46:20
9999996,5aca5a16-07df-4841-a46f-7ba41b94b899,869abc6e26dc43dd93351abd750efe6f,2022-09-12 13:46:25
9999997,b69e89d0-e865-46ce-aa7f-fe9c3078fe02,b0faa97003fb477bb76616dac3da198e,2022-09-12 13:46:30
9999998,1cddb993-ad5d-456e-b322-58a0520e7807,065b058e3b734a56ba5e17b2244bb47a,2022-09-12 13:46:35


проверяем дубликаты для покупателей и продуктов

In [ ]:
dups_customer_id = df.pivot_table(columns=['customer_id'], aggfunc='size')
dups_customer_id

In [ ]:
dups_product_id = df.pivot_table(columns=['product_id'], aggfunc='size')
dups_product_id

пишем функцию добавления сессий

In [ ]:
def add_session(group):
    group['deltaTsec'] = group["timestamp"].diff().dt.seconds.fillna(0).astype(int)
#     print(group)
    
    sessions = []
    session_time = 0
    session_limit = 180
    i = 0
    for delta in group['deltaTsec']:
        session_time += delta
        if i == 0 or session_time > session_limit:
            session_time = 0
            sessions.append(str(uuid.uuid4()))
        elif session_time <= session_limit:
            sessions.append(sessions[i - 1])
        
#         print("session_time",session_time)
        i += 1
        
    group['session_id'] = sessions
    group = group.drop('deltaTsec', axis=1)
#     print(group)
    return group

df_groupby = df.groupby("customer_id", as_index=False).apply(add_session)
df_groupby

In [ ]:
dups_session_id = df_groupby.pivot_table(columns=['session_id'], aggfunc='size')
dups_session_id

проверяем, перебирая сессии и проверяя, что разница между первым и последним `timestamp` для уникальной сессии составляет не более 3 минут

In [ ]:
def test(session_group):
    print("=====\nДля сессии", session_group.name, 
          "\nпользователя", session_group['customer_id'].iloc[0], 
          "\nразница составляет", session_group['timestamp'].iloc[-1] - session_group['timestamp'].iloc[0], 
          " - ", session_group['timestamp'].iloc[-1] - session_group['timestamp'].iloc[0] <= datetime.timedelta(seconds=180))

df_groupby.groupby("session_id", as_index=False).apply(test)